Домашнее задание

Реализовать обучение линейной регрессии для задачи boston house prices (https://www.kaggle.com/vikrishnan/boston-house-prices) с использованием torch’а

** Реализоыать наивный баесовский классификатор для MNIST (взяв всего 2 цифры “1” и “2”) сравнить с sclearn’овским


import libraries

In [16]:
import torch
from torch.utils.data import TensorDataset, DataLoader

from sklearn.datasets import load_boston
import pandas as pd 

helpers

In [5]:
def transform(data):
    data = torch.tensor(data.getdata(), dtype=torch.float32)
    return torch.floor(data/128.).squeeze(axis=1)

loading data

In [6]:
boston = load_boston()

In [23]:
data = boston['data']
target = boston['target']

In [24]:
data.shape, target.shape

((506, 13), (506,))